In [1]:
from gpt4all import GPT4All

In [2]:
model_name = "Phi-3-mini-4k-instruct.Q4_0.gguf"

model = GPT4All(model_name)

In [3]:
with model.chat_session():
    print(model.generate("What is 500 * 500"))

 The product of 500 multiplied by 500 is:

500 * 500 = 250,000.


In [4]:
model.retrieve_model(model_name)

{'order': 'q',
 'md5sum': 'f8347badde9bfc2efbe89124d78ddaf5',
 'name': 'Phi-3 Mini Instruct',
 'filename': 'Phi-3-mini-4k-instruct.Q4_0.gguf',
 'filesize': '2176181568',
 'requires': '2.7.1',
 'ramrequired': '4',
 'parameters': '4 billion',
 'quant': 'q4_0',
 'type': 'Phi-3',
 'description': '<ul><li>Very fast responses</li><li>Chat based model</li><li>Accepts system prompts in Phi-3 format</li><li>Trained by Microsoft</li><li>License: <a href="https://opensource.org/license/mit">MIT</a></li><li>No restrictions on commercial use</li></ul>',
 'url': 'https://gpt4all.io/models/gguf/Phi-3-mini-4k-instruct.Q4_0.gguf',
 'promptTemplate': '<|user|>\n{0}<|end|>\n<|assistant|>\n{1}<|end|>\n',
 'systemPrompt': '',
 'chatTemplate': "{{- bos_token }}\n{%- for message in messages %}\n    {{- '<|' + message['role'] + '|>\\n' + message['content'] + '<|end|>\\n' }}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|assistant|>\\n' }}\n{%- else %}\n    {{- eos_token }}\n{%- endif %}",
 'path'

In [10]:
with model.chat_session():
    print(model.generate('Hello, My Name is Osher'))
    print(model.generate('What is my name?'))

 Hi there! It's great to meet you. Although I don't have a name myself as an AI, feel free to call me Assistant. How can I help you today? If you like sharing your own experiences or asking questions, that would be wonderful too! Remember, everyone has their unique identity and story - just like yours is Osher.
 Hello again! As an AI, I don't have personal knowledge of individuals unless it was shared with me in a conversation for privacy reasons. However, you mentioned your name as "Osher" earlier during our interaction. Remember though that this information isn't stored or used beyond the current session to ensure confidentiality and respect your privacy.


In [10]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


pdf_loader = PyPDFLoader(fr'c:\\Users\\Osher N Boudara\\OneDrive\\Desktop_Ubuntu\\Projects\\Development\\resume_app\\resume\\osher_boudara_resume.pdf')

In [ ]:
pdf_loader.load()


[Document(metadata={'producer': 'Skia/PDF m134 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Osher_Boudara_Resume', 'source': 'c:\\\\Users\\\\Osher N Boudara\\\\OneDrive\\\\Desktop_Ubuntu\\\\Projects\\\\Development\\\\resume_app\\\\resume\\\\osher_boudara_resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="OSHER  BOUDARA  osherboudara99@gmail.com |  818.809.4261    \nEXPERIENCE  Cognizant   -   Data  Scientist  [ [ B R ] ]                                Aug.  2022  -  Present ●  Leading  and  developing  projects  in  multiple  data  science  and  engineering  teams  in  the  crop  science  division  of  a  global  \nFortune\n \n500\n \nclient\n \ncompany,\n \nadapting\n \nto\n \nevolving\n \nrequirements\n \nand\n \ndelivering\n \nsolutions\n \nto\n \nstakeholders.\n ●  Implementing  data  science  life  cycle  components:  automated  ETL  pipelines,  MLOps,  cloud-computing  code  development,  \ncloud\n \ntechnologies,\n \nversion\n